In [4]:
import nbimporter

# Import main functionality
from GPDM_direct_fixedpoints import *
from Experiment_1d_bifurcation_x3_varyslope import *

plotly.offline.init_notebook_mode(connected=True)

# Saving outputs and timing
import pickle, datetime, time

from joblib import Parallel, delayed
import multiprocessing
import itertools

Importing Jupyter notebook from Experiment_1d_bifurcation_x3_varyslope.ipynb


In [20]:
def save_csv(fname, arr):
    f_handle = open("../../Text/ICML2018/FIGURES/CSV/"+fname, 'wb')
    np.savetxt(f_handle, arr, delimiter=",")
    f_handle.close()

## Figure 2A - Show GP fit with training data

In [8]:
def bifurc_callback_plot_external(pvec_partial, 
                                  opt_params, init_paramvec, transforms, dict_ind, dict_shape,
                                  bifurc_params, trainingData=None
                                 ):
    
    paramvec = replace_params(pvec_partial, opt_params, init_paramvec)
    paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)
       
    # Unpack the usual parameters
    (Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J)  = \
        paramdict.values()[:12]
    
    if np.any(np.isnan(lengthscales)):
        set_trace()
    
    # Deal with the extra possible parameters
    Sigma_s = None; Sigma_J=None;
    if 'Sigma_s' in paramdict.keys():
        Sigma_s = paramdict['Sigma_s']
    if 'Sigma_J' in paramdict.keys():
        Sigma_J = paramdict['Sigma_J']
        
    # Plot transition function
    xstar = np.atleast_2d(np.arange(np.min(z)-0.5,np.max(z)+0.5,0.05))
    
    
    # Debugging stuff
#     z=z[:,:2]
#     u=u[:,:2]
#     Sigma_u=Sigma_u[:2,:]
#     print Sigma_J
    
    L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales, z=z, u=u, s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u=Sigma_u, sig_s=Sigma_s, sig_J = Sigma_J)
    mu_star, sig_star, K_pred = fp_predict(xstar, L, targets, params)

    # print(time_full_iter(pvec, y, dict_ind, dict_shape)[0])
        
    # Get true function values
    true_tr_vals = bifurc_trans_func(xstar, bifurc_params['slope'])
    
    #set_trace()
    
    # Map Sigma_s values to the range 8-20
    if Sigma_s.size>1:
        FP_SIZE = -np.squeeze(np.log(Sigma_s))
        FP_SIZE = FP_SIZE - np.min(FP_SIZE)
        FP_SIZE = (FP_SIZE/np.max(FP_SIZE))*12. + 8.
    else:
        FP_SIZE = 10.
    
    pltArray= [plt_type.Scatter(x=np.squeeze(xstar), y=np.squeeze(mu_star), mode='markers', name='GP post. mean',
                         marker=dict(color='blue')),
         plt_type.Scatter(x=np.squeeze(xstar), y=np.squeeze(mu_star)+2.*np.squeeze(np.sqrt(sig_star)), mode='markers', 
                         marker=dict(size=2, color='blue')),
         plt_type.Scatter(x=np.squeeze(xstar), y=np.squeeze(mu_star)-2.*np.squeeze(np.sqrt(sig_star)), mode='markers', 
                         marker=dict(size=2, color='blue')),      
         plt_type.Scatter(x=np.squeeze(xstar), 
                          y=np.squeeze(true_tr_vals), mode='markers', name = 'True trans. f.',
                          marker=dict(color='orange')),
         plt_type.Scatter(x=np.squeeze(xstar), 
                          y=np.squeeze(true_tr_vals+2.*np.sqrt(bifurc_params["Sigma_eps"])), mode='markers', name = 'True trans. f.',
                          marker=dict(color='orange', size=2)),
         plt_type.Scatter(x=np.squeeze(xstar), 
                          y=np.squeeze(true_tr_vals-2.*np.sqrt(bifurc_params["Sigma_eps"])), mode='markers', name = 'True trans. f.',
                          marker=dict(color='orange', size=2)),
         plt_type.Scatter(x=np.squeeze(z), y=np.squeeze(-2.0*np.ones_like(z)), mode='markers', marker=dict(size=10),
                         name = 'Ind point loc'),
         plt_type.Scatter(x=np.squeeze(z), y=np.squeeze(u), mode='markers', marker=dict(size=10),
                         name = 'Ind point val'),
         plt_type.Scatter(x=np.atleast_1d(np.squeeze(s)), y=np.atleast_1d(np.squeeze(s)), mode='markers', marker=dict(size=FP_SIZE),
                         name = 'Fixed point')
        ]
    
    if trainingData is not None:
        trainingData_t = np.reshape(trainingData[:,1:,:],(1,-1))
        trainingData_t1 = np.reshape(trainingData[:,:-1,:],(1,-1))
        pltArray.append(plt_type.Scatter(x=np.squeeze(trainingData_t1), y=np.squeeze(trainingData_t),
                                        mode='markers', marker=dict(size=4, color='black'),
                                         name='Training data'
                                        ))
    
    plt(pltArray)
    
    return pltArray

In [40]:
# Load the data set of interest
start_time = '20180208T082207'
expr_fname_params = "_varyslope_%0.3d" % (85)
load_fname_params = "_%0.2d_fix_%0.3d_trials_batch_%0.2d" % (5, 32, 0)
load_fname = "Experiment_1d_bifurc_results/bifurc_1d_" + start_time + load_fname_params + expr_fname_params + ".pkl"
    
curFit = pickle.load(open(load_fname, 'r'))

In [41]:
[y_train, x_train, bifurc_params,
 all_results, 
 init_paramvec, dict_ind, dict_shape, opt_params, 
 bnds, transforms] = curFit

paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)

fp_p_vals = fp_dynamic_threshold(curFit)
fp_size = fp_p_vals - np.min(fp_p_vals)
fp_size = fp_size/np.max(fp_size)
fp_size = np.squeeze(fp_size*16+4)
color_arr = np.array(['red', 'blue', 'black'])
color_choice = np.int32(np.squeeze(np.abs(paramdict['J'])>1.))


a = bifurc_callback_plot_external(all_results[-1].x, 
        opt_params, init_paramvec, transforms, dict_ind, dict_shape,
        bifurc_params, trainingData=y_train)


# We need the traces for training data, GP variances and fixed points

# Line1 - Training data (simply coordinates, do styling in latex)
save_csv("figure2_panel1_line1.csv", np.concatenate([a[-1].x[:,None],a[-1].y[:,None]],axis=1))

# Line2 - GP mean (simply coordinates, do styling in latex)
save_csv("figure2_panel1_line2.csv", np.concatenate([a[0].x[:,None],a[0].y[:,None]],axis=1))

# Line3 - GP +2STD (simply coordinates, do styling in latex)
save_csv("figure2_panel1_line3.csv", np.concatenate([a[1].x[:,None],a[1].y[:,None]],axis=1))

# Line4 - GP -2STD (simply coordinates, do styling in latex)
save_csv("figure2_panel1_line4.csv", np.concatenate([a[2].x[:,None],a[2].y[:,None]],axis=1))

# Line5 - Fixed point coordinates, sizes as 3rd column and (color as 4th column) # Python plot marker size: a[-2].marker.size[:,None]
save_csv("figure2_panel1_line5.csv", np.concatenate([a[-2].x[:,None],a[-2].y[:,None], fp_size[:,None], color_choice[:,None]],axis=1))

In [42]:
# Load the data set of interest
start_time = '20180208T082207'
expr_fname_params = "_varyslope_%0.3d" % (175)
load_fname_params = "_%0.2d_fix_%0.3d_trials_batch_%0.2d" % (5, 32, 0)
load_fname = "Experiment_1d_bifurc_results/bifurc_1d_" + start_time + load_fname_params + expr_fname_params + ".pkl"
    
curFit = pickle.load(open(load_fname, 'r'))

In [43]:
[y_train, x_train, bifurc_params,
 all_results, 
 init_paramvec, dict_ind, dict_shape, opt_params, 
 bnds, transforms] = curFit

paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)

fp_p_vals = fp_dynamic_threshold(curFit)
fp_size = fp_p_vals - np.min(fp_p_vals)
fp_size = fp_size/np.max(fp_size)
fp_size = np.squeeze(fp_size*16+4)
color_arr = np.array(['red', 'blue', 'black'])
color_choice = np.int32(np.squeeze(np.abs(paramdict['J'])>1.))


a = bifurc_callback_plot_external(all_results[-1].x, 
        opt_params, init_paramvec, transforms, dict_ind, dict_shape,
        bifurc_params, trainingData=y_train)


# We need the traces for training data, GP variances and fixed points

# Line1 - Training data (simply coordinates, do styling in latex)
save_csv("figure2_panel2_line1.csv", np.concatenate([a[-1].x[:,None],a[-1].y[:,None]],axis=1))

# Line2 - GP mean (simply coordinates, do styling in latex)
save_csv("figure2_panel2_line2.csv", np.concatenate([a[0].x[:,None],a[0].y[:,None]],axis=1))

# Line3 - GP +2STD (simply coordinates, do styling in latex)
save_csv("figure2_panel2_line3.csv", np.concatenate([a[1].x[:,None],a[1].y[:,None]],axis=1))

# Line4 - GP -2STD (simply coordinates, do styling in latex)
save_csv("figure2_panel2_line4.csv", np.concatenate([a[2].x[:,None],a[2].y[:,None]],axis=1))

# Line5 - Fixed point coordinates, sizes as 3rd column and (color as 4th column) # Python plot marker size: a[-2].marker.size[:,None]
save_csv("figure2_panel2_line5.csv", np.concatenate([a[-2].x[:,None],a[-2].y[:,None], fp_size[:,None], color_choice[:,None]],axis=1))

# Figure 2B - Show the bifurcation plot from noisy data

In [45]:
start_time='20180208T082207'
nbatches = 2
slope_set = np.array([0.5, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.45, 1.5, 1.75, 2.0, 2.25])
ntrials_set = np.array([32])
nfix_set = np.array([5])

GP_fit_saves = Parallel(n_jobs=1)(
    delayed(load_GP_fit)(slope_ind, ntrials_ind, nfix_ind, batchnum_ind, slope, ntrials, nfix, batchnum, start_time)
    for (slope_ind, ntrials_ind, nfix_ind, batchnum_ind), (slope, ntrials, nfix, batchnum) in (
        itertools.izip(
            itertools.product(range(len(slope_set)), range(len(ntrials_set)), range(len(nfix_set)), range(nbatches)),
            itertools.product(slope_set, ntrials_set, nfix_set, range(nbatches))
        )
    )
)

In [61]:
GP_fit_saves_rs = np.reshape(nestedlist_to_array(GP_fit_saves, max_level=0), 
                             (len(slope_set), len(ntrials_set), len(nfix_set), nbatches))

all_r_vals = np.zeros((0,1))
all_x_vals = np.zeros((0,1))
all_sizes = np.zeros((0,1))
all_color_inds = np.zeros((0,1))

for (slope_ind, slope, curFit) in itertools.izip(range(len(slope_set)), slope_set, GP_fit_saves_rs[:,0,0,0]):
    [y_train, x_train, bifurc_params,
     all_results, 
     init_paramvec, dict_ind, dict_shape, opt_params, 
     bnds, transforms] = curFit
    
    paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
    paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)
    
    # Relying on "how much this fixed points affects posterior"
    fp_p_vals = fp_dynamic_threshold(curFit)
    color_arr = np.array(['red', 'blue', 'black'])
    color_choice = np.int32(np.squeeze(np.abs(paramdict['J'])>1.))

    # Renormalise (linearly) marker sizes between 4 and 20 pts
    fp_size = fp_p_vals - np.min(fp_p_vals)
    fp_size = fp_size/np.max(fp_size)
    fp_size = np.squeeze(fp_size*16+4)
    

    
    all_r_vals = np.concatenate([all_r_vals, slope*np.ones((paramdict['Sigma_s'].shape[0],1))], axis=0)
    all_x_vals = np.concatenate([all_x_vals, paramdict['s'].T], axis=0)
    all_sizes = np.concatenate([all_sizes, fp_size[:,None]], axis=0)
    all_color_inds = np.concatenate([all_color_inds, color_choice[:,None]], axis=0)

In [62]:
# Line5 - Fixed point coordinates, sizes as 3rd column and (color as 4th column) # Python plot marker size: a[-2].marker.size[:,None]
save_csv("figure2_panel3_line1.csv", np.concatenate([all_r_vals, all_x_vals, all_sizes, all_color_inds],axis=1))